In [6]:
import pandas as pd
import requests
#we use beautiful soup for web scraping
from bs4 import BeautifulSoup

In [43]:
#function to return data frame of one page by web scrapping
fdf = pd.DataFrame()

def fetchDataByWebScrapping(pageNo):
    #we send headers to make the server think like this request is coming from a browser
    headers = {'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Mobile Safari/537.36'}
    webpage = requests.get("https://www.ambitionbox.com/list-of-companies?page={}".format(pageNo),headers=headers).text
    soup = BeautifulSoup(webpage,'lxml')
    #print(soup.prettify())
    #now we can find specific things in web page using soup.find_all(HTML tag name,class_='classname'(if any))
    company = soup.find_all('div',class_='companyCardWrapper')
    # len(company) #20 listings on each page
    #now extracting all info using loop
    name =[]
    rating = []
    service = []
    remark_div = []
    highlyRatedFor=[]   # to extract high from rating_div
    badlyRatedFor=[]  # to extract critical from rating_div
    for i in company:
        name.append(i.find('h2').text.strip())#now we are using text.strip() to remove all \n and other tab spaces in div
        rating.append(i.find('span',class_='companyCardWrapper__companyRatingValue').text.strip())
        service.append(i.find('span',class_='companyCardWrapper__interLinking').text.strip())
        remark_div.append(i.find('div',class_='companyCardWrapper__ratingComparisonWrapper')) # we can use find_all along with extend also //PS- wasted 1 hr

    # now seperating high and critical
    for di in remark_div:
        if di is None:
            highlyRatedFor.append('NaN')
            badlyRatedFor.append('NaN')
            continue
        pos = di.find('span',class_='companyCardWrapper__ratingHeader--high')
        if pos:
            pos_value = pos.find_next('span',class_='companyCardWrapper__ratingValues')
            if pos_value:
                highlyRatedFor.append(pos_value.text.strip())
        else:
            highlyRatedFor.append('NaN')
        neg = di.find('span',class_='companyCardWrapper__ratingHeader--critical')
        if neg:
            neg_value = neg.find_next('span',class_='companyCardWrapper__ratingValues')
            if neg_value:
                badlyRatedFor.append(neg_value.text.strip())
        else:
            badlyRatedFor.append('NaN')
        #print(highlyRatedFor)
        #print(badlyRatedFor)
        # now we will create a data frame
    data = {'name':name,'rating':rating,'service':service,'highlyRatedFor':highlyRatedFor,'badlyRatedFor':badlyRatedFor}
    df = pd.DataFrame(data)
    df[['Industry', 'Location']] = df['service'].str.extract(r'([^|]+)\s*\|\s*([^\+]+)')
    df.drop('service',axis=1,inplace=True)
    return df

In [44]:
#calling function in loop
for pageNo in range(1,11):
    fdf = pd.concat([fdf,fetchDataByWebScrapping(pageNo)],ignore_index=True)

In [45]:
fdf.shape

(200, 6)

In [46]:
fdf.head()

,name,rating,highlyRatedFor,badlyRatedFor,Industry,Location
0,TCS,3.7,"Job Security, Work Life Balance","Promotions / Appraisal, Salary & Benefits, Wor...",IT Services & Consulting,Mumbai
1,Accenture,3.9,"Company Culture, Job Security, Skill Developme...",Promotions / Appraisal,IT Services & Consulting,Dublin
2,Cognizant,3.8,NaN,Promotions / Appraisal,IT Services & Consulting,Teaneck. New Jersey.
3,Wipro,3.7,Job Security,"Promotions / Appraisal, Salary & Benefits",IT Services & Consulting,Bangalore/Bengaluru
4,Capgemini,3.8,"Job Security, Work Life Balance, Skill Develop...","Promotions / Appraisal, Salary & Benefits",IT Services & Consulting,Paris


In [48]:
fdf.to_csv('web-scraping-listOfCompanies-ambitionBox.csv')